# Head Velocity
Notebook for clean attempt of checking the distribution of head velocity including the direction.

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats, integrate, signal

%matplotlib inline
sns.set(color_codes=True)

# loading data
df = {}
event = {}

fname = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC', ]
                                                                            
path = '../datasets/'

for i, x in enumerate(fname):
    df[i] = pd.read_hdf(path+x+'_filter.h5', 'Full')
    event[i] = pd.read_hdf(path+'analysis/'+x+'_event.h5', 'Event')      
                                                                                                                                    

In [2]:
# in use functions
def angle_cos(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    return CosTheta


def angle_zero(df):
    V1 = np.array([df.lx1, 0, df.lz1])
    V2 = np.array([1     , 0, 0     ])
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    df['theta0'] = np.arccos(CosTheta)
    return df


def angle_orig(df):
    V1 = np.array([df.X_Ori, df.Y_Ori, df.Z_Ori])
    V2 = np.array([1       , 0       , 0     ])
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    df['thetaOrig'] = np.arccos(CosTheta)
    return df


def decompose_vec_df(df):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([df.X_Ori, df.Y_Ori, df.Z_Ori])  
    Vz = np.array([df.X_Ori, df.Y_Ori, 0       ])  
    Vx = np.array([0       , df.Y_Ori, df.Z_Ori])

    # find the angles between them and orientation vector 
    df['lx1'] = angle_cos(Vo, Vz)
    df['lz1'] = angle_cos(Vo, Vx)
    return df


## All experiments analysis
Mergning event log and filetered data from experiments. Removing useless columns. Calculating the flattened vector in XZ plane (lx1, lz1) - including sing, and normalization to unitvector lenght. 

In [3]:
DF = {}

for i, x in enumerate(fname):
    DF[i] = pd.merge(event[i], df[i], on='Frame')

    #clearing data off useless variables
    DF[i].rename(columns={"Time_Ori":'Time'}, inplace=True)
    DF[i].drop({'Time_Pos', 'X_Pos', 'Y_Pos', 'Z_Pos', 'dX', 'dY', 'dZ',}, axis=1, inplace=True)
    print('done %d' % i)

done 0
done 1
done 2
done 3
done 4
done 5
done 6
done 7


In [ ]:
# calculating lenghts of VecX and VecZ and creation of shifted vectors
for i, x in enumerate(fname):
    DF[i] = DF[i].apply(decompose_vec_df, axis=1)
    DF[i]['V1'] = np.linalg.norm(np.array([DF[i].lx1, 0, DF[i].lz1]))

    # propagate the sign of a X, Z to new vector
    DF[i]['lx1'] = DF[i].apply(lambda row: row.lx1 * (row.X_Ori / np.absolute(row.X_Ori)), axis=1)
    DF[i]['lz1'] = DF[i].apply(lambda row: row.lz1 * (row.Z_Ori / np.absolute(row.Z_Ori)), axis=1)

    # normalize the data to unit vector form
    DF[i]['lx1'] = DF[i].apply(lambda row: row.lx1 / row.V1, axis=1)
    DF[i]['lz1'] = DF[i].apply(lambda row: row.lz1 / row.V1, axis=1)

    DF[i].dropna(inplace=True)
    print('done %d' % i)

done 0
done 1
done 2
done 3
done 4
done 5
done 6
done 7


## Angle and Velocity
Calculating the angle between orientation 2D vector and origin set to XZ(1,0). Substracting the angles between iterations to calculate change of angle in each iteration. Based on that: dV = dTheta / dTime

In [ ]:
# calculatinig the angles between X axis and the vectors
for i, x in enumerate(fname):
    DF[i] = DF[i].apply(angle_orig, axis=1) # calculation between two vectors(does not include direction) -  theta
    DF[i] = DF[i].apply(angle_zero, axis=1) # calculation with origin

    DF[i]['theta1'] = DF[i].theta0.shift(1)
    DF[i]['theta']  = DF[i].apply(lambda row: row.theta0 - row.theta1, axis=1)
    DF[i].drop({'theta0','theta1'}, axis=1, inplace=True)
    print('done %d' % i)

done 0


In [ ]:
# angular velocity
for i, x in enumerate(fname):
    DF[i]['U']     = DF[i].apply(lambda row: np.degrees(row.theta)     / row.dTime, axis=1) #between two vectors velocity
    DF[i]['UOrig'] = DF[i].apply(lambda row: np.degrees(row.thetaOrig) / row.dTime, axis=1)
    DF[i].dropna(inplace=True)
    print('done %d' % i)

In [ ]:
# U and theta distributions

fig = plt.figure(figsize=(9,9))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
for i, x in enumerate(fname):

    sns.distplot(DF[i].theta, ax=ax1)
    sns.distplot(DF[i].U    , ax=ax2)
    c2 = ax3.plot(DF[i].Time, DF[i].U)


## Filtering velocity values

In [ ]:
# U and theta distributions with limited velocity
vlim = 100

for i, x in enumerate(fname):
    DF[i] = DF[i][DF[i]['dTime'] < 0.1]
    DF[i] = DF[i][DF[i]['U'] < vlim]
    DF[i] = DF[i][DF[i]['U'] >-vlim]
    print('done %d' % i)

# U and theta distributions
fig = plt.figure(figsize=(9,9))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
for i, x in enumerate(fname):

    sns.distplot(DF[i].theta, ax=ax1)
    sns.distplot(DF[i].U    , ax=ax2)
    c2 = ax3.plot(DF[i].Time, DF[i].U)

## Angular velocity analysis
Comparison of velocity logged per even with calculated velocit (substraction and correlation). Addtional smoothing of the data applied.

In [ ]:
for i, x in enumerate(fname):
    # smoothing the data
    DF[i]['M']     = DF[i]['U'].rolling(window=50).mean()

    # substraction
    DF[i]['sub1']  = DF[i].apply(lambda row: row.speed - row.U, axis=1)
    DF[i]['sub2']  = DF[i].apply(lambda row: row.speed - row.M, axis=1)

    # correlation
    DF[i]['corr']  = signal.correlate(DF[i].speed, DF[i].U, mode='same') / 128
    DF[i]['corrM'] = signal.correlate(DF[i].speed, DF[i].M, mode='same') / 128
    print('done %d' % i)
    

In [ ]:
for i, x in enumerate(fname):
    ## Plotting
    fig = plt.figure(figsize=(15,12))
    plt.title('experiment %d' % i)
    ax1 = fig.add_subplot(511)
    ax2 = fig.add_subplot(512)
    ax3 = fig.add_subplot(513)
    ax4 = fig.add_subplot(514)
    ax5 = fig.add_subplot(515)

    c1 = ax1.plot(DF[i].Time, DF[i].U)
    c2 = ax2.plot(DF[i].Time, DF[i].speed)
    c3 = ax3.plot(DF[i].Time, DF[i].M)
    c3 = ax4.plot(DF[i].Time, DF[i].speed, DF[i].Time, DF[i].sub1)
    c5 = ax5.plot(DF[i].Time, DF[i].speed, DF[i].Time, DF[i].sub2)


In [ ]:
for i, x in enumerate(fname):
    ## Plotting
    fig = plt.figure(figsize=(15,15))
    plt.title('experiment %d' % i)
    ax1 = fig.add_subplot(511)
    ax2 = fig.add_subplot(512)
    ax3 = fig.add_subplot(513)
    ax4 = fig.add_subplot(514)
    ax5 = fig.add_subplot(515)

    r = range(0, len(DF[i].Time))
    
    c1 = ax1.plot(r, DF[i].U)
    c2 = ax2.plot(r, DF[i].speed)
    c3 = ax3.plot(r, DF[i].M)
    c3 = ax4.plot(r, DF[i].speed, r, DF[i].sub1)
    c5 = ax5.plot(r, DF[i].speed, r, DF[i].sub2)
    

## Stimulus-related behavior
Looking for frames detected as SRB. According to:
SRB = {f E FRAMES: ||Vhead - Vstim|| < Dmax

In [ ]:
# creation of needed dataset
dfSRB = {}

for i, x in enumerate(fname):
    dfSRB[i] = DF[i].copy()
    dfSRB[i].drop({'MotiveExpTimeSecs', 'X_Ori', 'Y_Ori', 'Z_Ori', 'dTime', 'Vxyz', 'lx1', 'lz1', 'V1',
                'thetaOrig', 'theta', 'UOrig', 'M', 'corr','corrM', 'subU', }, axis=1, inplace=True)

    dfSRB[i].dropna(inplace=True)


In [ ]:
sumSRB, dfSRBVis, dfSRBInvis, sumSRBVis, sumSRBInvis = {}, {}, {}, {}, {}

for i, x in enumerate(fname):
    # setting Dmax as 20% of velocity at given moment // what about 0
    dfSRB[i]['Dmax'] = dfSRB[i].apply(lambda row: row.speed * 0.2, axis=1)
    dfSRB[i]['SRB']  = dfSRB[i].apply(lambda row: np.absolute(row.subM) < np.absolute(row.Dmax), axis=1)

    # checking lenght of SRB frames in the filtered experiment data
    sumSRB[i] = len(dfSRB[i][dfSRB[i]['SRB']])

    #  temp dataframes with stimuli being present or not
    dfSRBVis[i]   = dfSRB[i][ dfSRB[i]['visible']]
    dfSRBInvis[i] = dfSRB[i][~dfSRB[i]['visible']]

    sumSRBVis[i]  = len(dfSRBVis[i]  [dfSRBVis[i]['SRB']])
    sumSRBInvis[i]= len(dfSRBInvis[i][dfSRBInvis[i]['SRB']])
 
    print('exp %d; stimulus-related behavior: %f%%'     % (i, sumSRB[i]      * 100 / len(dfSRB[i])     ))
    print('exp %d; SRM when stimuli is visible: %f%%'   % (i, sumSRBVis[i]   * 100 / len(dfSRBVis[i])  ))
    print('exp %d; SRM when stimuli is invisible: %f%%' % (i, sumSRBInvis[i] * 100 / len(dfSRBInvis[i])))

#print('\n', sumSRB[i])
#pd.set_option('display.max_rows', 2000)
#print(fname[i])
#print(dfSRB[dfSRB['SRB']])